In [1]:
!pip install bs4
!pip install contractions
!pip install scikit-multilearn
!pip install keras-tuner

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=1345c083723877f17033173e3852ace590829cabd9e0bdb36bb8bc28f8554573
  Stored in directory: /home/andres/.cache/pip/wheels/e4/62/1d/d4d1bc4f33350ff84227f89b258edb552d604138e3739f5c83
Successfully built bs4
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 1.1 MB/s eta 0:00:001m930.5 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 886.1 kB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 1.6 MB/s eta 0:00:0011.0 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 674.1 kB/s eta 0:00:00m eta 0:00:010:00:01


In [3]:
!pip install spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 852.2 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 980.2 kB/s eta 0:00:00MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 913.3/913.3 kB 716.7 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.7/124.7 kB 833.6 kB/s eta 0:00:00 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.2/492.2 kB 492.0 kB/s eta 0:00:001m476.1 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 1.2 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 1.1 MB/s eta 0:00:00m eta 0:00:0136m0:00:010m


In [6]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
import contractions
import spacy
import nltk
import re
import pickle
import glob
nltk.download('punkt')
%matplotlib inline

[nltk_data] Downloading package punkt to /home/andres/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
!python -m spacy download en_core_web_sm

2023-06-01 11:45:00.266636: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 1.1 MB/s eta 0:00:00m eta 0:00:010:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [9]:
sp = spacy.load("en_core_web_sm")
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [12]:
dataFrameTrain = pd.read_csv('../Data/train.csv')
dataFrameTrain['clear'] = (dataFrameTrain[dataFrameTrain.columns[1:7]].max(axis=1) == 0).astype(int)

/tmp/ipykernel_43130/3693549209.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  dataFrameTrain['clear'] = (dataFrameTrain[dataFrameTrain.columns[1:7]].max(axis=1) == 0).astype(int)


In [13]:
dataFrameTrain

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clear
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,1
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,1
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,1
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,1
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,1
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,1
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,1
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0,1


In [14]:
#Text normalization
import re
pd.options.mode.chained_assignment = None
dataFrameTrain['comment_text'] = dataFrameTrain['comment_text'].apply(lambda x: x.lower())
dataFrameTrain['comment_text'] = dataFrameTrain['comment_text'].apply((lambda x:re.sub('@[^\s]+','',x)))
dataFrameTrain['comment_text'] = dataFrameTrain['comment_text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

In [15]:
def tokenize(text):
    lemma = []
    pos = []
    explain = []
    lemma_stop = []
    pos_stop = []
    explain_stop = []
   
    for word in sp(text):     
        if not word.is_punct and word.text != ' ' and word.pos_ != 'SPACE':
            lemma.append(word.lemma_)
            pos.append(word.pos_)
            explain.append(spacy.explain(word.tag_))
            if not word.text in spacy_stopwords:
                lemma_stop.append(word.lemma_)
                pos_stop.append(word.pos_)
                explain_stop.append(spacy.explain(word.tag_))
                
            
    return lemma, pos, explain, lemma_stop, pos_stop, explain_stop
            
    return tokens
def remove_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text
def strip_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text()
    return stripped_text

def clean_text(text):
    text = contractions.fix(text)
    text = text.lower()
    text = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', '', text) # clean url
    text = re.sub(r'#(\w+)', '', text)   # clean hashes
    text = re.sub(r'@(\w+)', '', text)   # clean @
    text = re.sub(r'\d+', '', text)      # clean digits
    text = re.sub(r'[,!@\'\"?\.$%_&#*+-:;]', '', text)   # clean punctuation
    text = remove_accented_chars(text)
    text = strip_html_tags(text)
    
    return text    


In [ ]:
lemma_s = []
pos_s = []
explain_s = []
lemma_stop_s = []
pos_stop_s = []
explain_stop_s = []
ids_rem = []
count = 1
for i, e in zip(dataFrameTrain['comment_text'], range(dataFrameTrain['comment_text'].shape[0])):  
    if count % 20000 == 0:
        print(count)
    text  =  clean_text(i)
    
    lemma, pos, explain, lemma_stop, pos_stop, explain_stop = tokenize(text)
    if lemma != []:
        lemma_s.append(lemma)
        pos_s.append(pos)
        explain_s.append(explain)
        
    if lemma_stop != []:
        lemma_stop_s.append(lemma_stop)
        pos_stop_s.append(pos_stop)
        explain_stop_s.append(explain_stop)
    else:
        dataFrameTrain.drop(e, inplace=True)    
    count+=1

/home/andres/anaconda3/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(
/home/andres/anaconda3/lib/python3.10/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


In [ ]:
with open("../Data/processed/sentences_lemma.data", 'wb') as filehandle:
    pickle.dump(lemma_s, filehandle)
with open("../Data/processed/sentences_pos.data", 'wb') as filehandle:
    pickle.dump(pos_s, filehandle)
with open("../Data/processed/sentences_explain.data", 'wb') as filehandle:
    pickle.dump(explain_s, filehandle)
with open("../Data/processed/sentences_lemma_stop.data", 'wb') as filehandle:
    pickle.dump(lemma_stop_s, filehandle)
with open("../Data/processed/sentences_pos_stop.data", 'wb') as filehandle:
    pickle.dump(pos_stop_s, filehandle)
with open("../Data/processed/sentences_explain_stop.data", 'wb') as filehandle:
    pickle.dump(explain_stop_s, filehandle)

In [ ]:
dataFrameTrain['lemma_stop'] = lemma_stop_s
dataFrameTrain['pos_stop'] = pos_stop_s
dataFrameTrain['explain_stop'] = explain_stop_s

In [ ]:
dataFrameTrain.to_csv('../Data/processed/tokenized_stop.csv', index=False)

In [ ]:
tokenized = pd.read_csv('../Data/processed/tokenized_stop.csv')
tokenized['lemma_stop'] = tokenized['lemma_stop'].apply(eval)
tokenized['pos_stop'] = tokenized['pos_stop'].apply(eval)
tokenized['explain_stop'] = tokenized['explain_stop'].apply(eval)


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
MAX_NB_WORDS = 20000
max_len = 200
X = tokenized['lemma_stop'].values
y = tokenized[tokenized.columns[2:8]].values

In [ ]:
from skmultilearn.model_selection import iterative_train_test_split
X_train, y_train, X_test, y_test = iterative_train_test_split(X.reshape(-1,1), y, test_size = 0.2)
X_train = np.squeeze(X_train)
X_test = np.squeeze(X_test)


In [ ]:
import joblib
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True, char_level=False)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
X_train = pad_sequences(X_train, padding='pre', truncating='pre', maxlen=max_len)
X_test = pad_sequences(X_test, padding='pre', truncating='pre', maxlen=max_len)
joblib.dump(tokenizer, 'Tokenizer/tok.joblib')

In [ ]:
word2idx = tokenizer.word_index
print('Found %s unique tokens.' % len(word2idx))

In [ ]:
train = pd.DataFrame()
train['X'] = X_train.tolist()
train['y'] = y_train.tolist()
test = pd.DataFrame()
test['X'] = X_test.tolist()
test['y'] = y_test.tolist()
train.to_csv('../Data/processed/train.csv', index=False)
test.to_csv('../Data/processed/test.csv', index=False)